In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [2]:
loader=PyPDFDirectoryLoader("./pdfs")
documents=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_doc=text_splitter.split_documents(documents)
final_doc[0]

Ignoring wrong pointing object 1367 0 (offset 0)
Ignoring wrong pointing object 1368 0 (offset 0)


Document(metadata={'source': 'pdfs\\seerat.pdf', 'page': 0}, page_content='MSA NIU  \n1 \nAr-Raheeq Al -Makhtum \n(THE SEALED NECTAR) \nMemoirs of the Noble Prophet [pbuh] \nby Saifur Rahman al -Mubarakpuri \nJamia Salafia - India Translated by Issam Diab  Maktaba Dar -us-Salam Publishers \nand Distributors Saudi Arabia§ UK § USA § Pakistan  \nTABLE OF CONTENT S  \nPublisher Note  \nAuthor’s Note  \nCertificate   \nLocation and nature of Arab Tribes ___________________________________ 8  \nLocation of the Arabs  \nArab Tribes   \nRulership and Princeship among the Arabs _____________________________ 14   \nRulership in Yemen  \nRulership in Heerah  \nRulership in Geographical Syria  \nRulership in Hijaz  \nThe Reasons of this war have been illustrated in three versions  \nRulership in Pan -Arabia  \nThe political situation  \n            Religions of the Arabs _____________________________________________ 22  \nThe Religious situation   \nAspects of Pre -Islamic Arabian Society _______

In [3]:
len(final_doc)

1286

In [4]:
hf_embeddings=HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

c:\Users\Lenovo\ML Notebooks\RAG\v_env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
vectorstor=FAISS.from_documents(final_doc,hf_embeddings)

In [8]:

## Query using Similarity Search
query="What was the situation of Arab during prophet Mohammad?"
relevant_docments=vectorstor.similarity_search(query)

In [9]:
print(relevant_docments[0].page_content)

The Religious situation   
Aspects of Pre -Islamic Arabian Society _______________________________ 28  
Social life of the Arabs  
The Economic Situati on  
Ethics   
The Lineage and Family of Muhammad [pbuh] _________________________ 32  
The prophetic Family


In [10]:
retriever=vectorstor.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [24]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()

# Retrieve the Hugging Face token
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["HF_TOKEN"] = hf_token


# This will prompt you to enter your token
login()


In [38]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300,"use_auth_token": hf_token}
)

llm = hf 
# llm.invoke(query)

KeyboardInterrupt: 

In [36]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    model_kwargs={"temperature":0.001,"max_length":500}

)
# Can not use temperature 0, so I have to use locally where I can use temperature 0

In [37]:
hf.invoke("Who are you?")

'Who are you? (2023)\nI am a human being, a member of the species Homo sapiens. I am a thinking, feeling, and experiencing individual, capable of perceiving the world around me through my senses. I am a unique and complex entity, with my own thoughts, emotions, and experiences that shape my identity and inform my interactions with others.\nI am a product of my environment, shaped by the cultural, social, and historical contexts in which I live. My identity is influenced by my family'